 
**Sentiment Analysis:**<br> 
The process of computationally identifying and categorizing opinions expressed in a piece of text, especially in order to determine whether the writer's attitude towards a particular topic, product, etc. is positive, negative, or neutral. In common ML words its just a classification problem. 

**What is class imbalance:**<br>
It is the problem in machine learning where the total number of a class of data (positive) is far less than the total number of another class of data (negative). This problem is extremely common in practice and can be observed in various disciplines including fraud detection, anomaly detection, medical diagnosis, oil spillage detection, facial recognition, etc.


**Solving class imbalanced data:**<br>
I am using the two most effective ways to mitigate this:<br>
- Up sampling 
- Using class weighted loss function

**Dataset**<br>
First GOP Debate Twitter Sentiment
About this Dataset
This data originally came from [Crowdflower's Data for Everyone library ](http://www.crowdflower.com/data-for-everyone).

> As the original source says,
> We looked through tens of thousands of tweets about the early August GOP debate in Ohio and asked contributors to do both
> sentiment analysis and data categorization. Contributors were asked if the tweet was relevant, which candidate was mentioned,
> what subject was mentioned, and then what the sentiment was for a given tweet. We've removed the non-relevant messages from
> the uploaded dataset.

**Details about model**<br>
 - model contains 3 layers (Embedding, LSTM, Dense with softmax).
 - Up-sampling is used to balance the data of minority class.
 - Loss function with different class weight in keras to further reduce class imbalance.



### Importing useful packages
Lets first import all libraries. Please make sure that you have these libraries installed.   

In [1]:

import numpy as np 
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,classification_report
import re

Using TensorFlow backend.
c:\users\asif\anaconda3\envs\tensor_experiment\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\asif\anaconda3\envs\tensor_experiment\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\asif\anaconda3\envs\tensor_experiment\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\asif\

### Data Preprocessing
- reading the data
- kepping only neccessary columns
- droping "Neutral" sentiment data


In [2]:
data = pd.read_csv("E:/Python Imarticus/Exam-Deep Learning & UnSuperVised Learning/Sentiment.csv")
# Keeping only the neccessary columns
data = data[['text','sentiment']]
data = data[data.sentiment != "Neutral"]

Let See the few lines of the data

In [4]:
data.head()

,text,sentiment
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",Positive
6,RT @warriorwoman91: I liked her and was happy ...,Negative


> A few things to notice here
- "RT @..." in start of every tweet
- a lot of special characters <br>
> We have to remove all this noise also lets convert text into lower case.


In [5]:
data['text'] = data['text'].apply(lambda x: x.lower())
# removing special chars
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
data['text'] = data['text'].str.replace('rt','')
data.head()

,text,sentiment
1,scottwalker didnt catch the full gopdebate la...,Positive
3,robgeorge that carly fiorina is trending hou...,Positive
4,danscavino gopdebate w realdonaldtrump delive...,Positive
5,gregabbott_tx tedcruz on my first day i will ...,Positive
6,warriorwoman91 i liked her and was happy when...,Negative


This looks better.<br>
Lets pre-process the data so that we can use it to train the model
- Tokenize
- Padding (to make all sequence of same lengths)
- Converting sentiments into numerical data(One-hot form)
- train test split


In [6]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(8583, 28) (8583, 2)
(2146, 28) (2146, 2)


### Defining model
Next, I compose the LSTM Network. Note that **embed_dim**, **lstm_out**, **batch_size**, **droupout_x** variables are hyper parameters, their values are somehow intuitive, can be and must be played with in order to achieve good results. Please also note that I am using softmax as activation function. The reason is that our Network is using categorical crossentropy, and softmax is just the right activation method for that.

In [7]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 28, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 28, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


### Building LSTM Model
Here we train the Network. We should run much more than 15 epoch, but I would have to wait forever (run more epochs later), so it is 15 for now. you will see progress bar (if you want to shut it up use verbose = 0)


In [8]:
batch_size = 128
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 1)


Epoch 1/15
8583/8583 [==============================] - 17s 2ms/step - loss: 0.4749 - accuracy: 0.7984
Epoch 2/15
8583/8583 [==============================] - 15s 2ms/step - loss: 0.3361 - accuracy: 0.8554
Epoch 3/15
8583/8583 [==============================] - 15s 2ms/step - loss: 0.2890 - accuracy: 0.8814
Epoch 4/15
8583/8583 [==============================] - 16s 2ms/step - loss: 0.2665 - accuracy: 0.8906
Epoch 5/15
8583/8583 [==============================] - 16s 2ms/step - loss: 0.2518 - accuracy: 0.8981
Epoch 6/15
8583/8583 [==============================] - 16s 2ms/step - loss: 0.2315 - accuracy: 0.9046
Epoch 7/15
8583/8583 [==============================] - 15s 2ms/step - loss: 0.2133 - accuracy: 0.9134
Epoch 8/15
8583/8583 [==============================] - 15s 2ms/step - loss: 0.2111 - accuracy: 0.9089
Epoch 9/15
8583/8583 [==============================] - 18s 2ms/step - loss: 0.1905 - accuracy: 0.9235
Epoch 10/15
8583/8583 [==============================] - 20s 2ms/step - 

### Let evaluate the model


In [18]:
Y_pred = model.predict_classes(X_test,batch_size = batch_size)
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))s

confusion matrix [[1365  334]
 [ 125  322]]
              precision    recall  f1-score   support

           0       0.92      0.80      0.86      1699
           1       0.49      0.72      0.58       447

    accuracy                           0.79      2146
   macro avg       0.70      0.76      0.72      2146
weighted avg       0.83      0.79      0.80      2146




## Solving data imbalance problem

**Up-sample Minority Class**

Up-sampling is the process of randomly duplicating observations from the minority class in order to reinforce its signal. There are several heuristics for doing so, but the most common way is to simply re-sample with replacement.

It's important that we separate test set before up-sampling because after up-sampling there will be multiple copies of same data point and if we do train test split after up-sampling the test set will not be completely unseen.


In [10]:
# Separate majority and minority classes
data_majority = data[data['sentiment'] == 'Negative']
data_minority = data[data['sentiment'] == 'Positive']

bias = data_minority.shape[0]/data_majority.shape[0]
# lets split train/test data first then 
train = pd.concat([data_majority.sample(frac=0.8,random_state=200),
         data_minority.sample(frac=0.8,random_state=200)])
test = pd.concat([data_majority.drop(data_majority.sample(frac=0.8,random_state=200).index),
        data_minority.drop(data_minority.sample(frac=0.8,random_state=200).index)])

train = shuffle(train)
test = shuffle(test)

In [11]:
print('positive data in training:',(train.sentiment == 'Positive').sum())
print('negative data in training:',(train.sentiment == 'Negative').sum())
print('positive data in test:',(test.sentiment == 'Positive').sum())
print('negative data in test:',(test.sentiment == 'Negative').sum())


positive data in training: 1789
negative data in training: 6794
positive data in test: 447
negative data in test: 1699


Now Lets do up-sampling

In [12]:
# Separate majority and minority classes in training data for upsampling 
data_majority = train[train['sentiment'] == 'Negative']
data_minority = train[train['sentiment'] == 'Positive']

print("majority class before upsample:",data_majority.shape)
print("minority class before upsample:",data_minority.shape)

# Upsample minority class
data_minority_upsampled = resample(data_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples= data_majority.shape[0],    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
data_upsampled = pd.concat([data_majority, data_minority_upsampled])
 
# Display new class counts
print("After upsampling\n",data_upsampled.sentiment.value_counts(),sep = "")

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values) # training with whole data

X_train = tokenizer.texts_to_sequences(data_upsampled['text'].values)
X_train = pad_sequences(X_train,maxlen=29)
Y_train = pd.get_dummies(data_upsampled['sentiment']).values
print('x_train shape:',X_train.shape)

X_test = tokenizer.texts_to_sequences(test['text'].values)
X_test = pad_sequences(X_test,maxlen=29)
Y_test = pd.get_dummies(test['sentiment']).values
print("x_test shape", X_test.shape)

majority class before upsample: (6794, 2)
minority class before upsample: (1789, 2)
After upsampling
Negative    6794
Positive    6794
Name: sentiment, dtype: int64
x_train shape: (13588, 29)
x_test shape (2146, 29)


In [13]:
# model
embed_dim = 128
lstm_out = 192

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X_train.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.4, recurrent_dropout=0.4))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 29, 128)           256000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 29, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 192)               246528    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 386       
Total params: 502,914
Trainable params: 502,914
Non-trainable params: 0
_________________________________________________________________
None


Lets define class weights as a dictionary, I have defined weight of majority class to be 1 and of minority class to be a multiple of 1/bias


In [14]:
batch_size = 128
# also adding weights
class_weights = {0: 1 ,
                1: 1.6/bias }
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 1,
          class_weight=class_weights)

Epoch 1/15
13588/13588 [==============================] - 27s 2ms/step - loss: 1.2557 - accuracy: 0.5741
Epoch 2/15
13588/13588 [==============================] - 26s 2ms/step - loss: 0.8070 - accuracy: 0.7501
Epoch 3/15
13588/13588 [==============================] - 27s 2ms/step - loss: 0.6680 - accuracy: 0.8140
Epoch 4/15
13588/13588 [==============================] - 28s 2ms/step - loss: 0.5893 - accuracy: 0.8372
Epoch 5/15
13588/13588 [==============================] - 23s 2ms/step - loss: 0.5367 - accuracy: 0.8518
Epoch 6/15
13588/13588 [==============================] - 31s 2ms/step - loss: 0.4889 - accuracy: 0.8669
Epoch 7/15
13588/13588 [==============================] - 31s 2ms/step - loss: 0.4603 - accuracy: 0.8777
Epoch 8/15
13588/13588 [==============================] - 40s 3ms/step - loss: 0.4429 - accuracy: 0.8795
Epoch 9/15
13588/13588 [==============================] - 29s 2ms/step - loss: 0.4085 - accuracy: 0.8920
Epoch 10/15
13588/13588 [==============================

In [15]:
Y_pred = model.predict_classes(X_test,batch_size = batch_size)
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

confusion matrix [[1329  370]
 [ 115  332]]
              precision    recall  f1-score   support

           0       0.92      0.78      0.85      1699
           1       0.47      0.74      0.58       447

    accuracy                           0.77      2146
   macro avg       0.70      0.76      0.71      2146
weighted avg       0.83      0.77      0.79      2146



In [16]:
# running model to few more epochs
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 1,
          class_weight=class_weights)
Y_pred = model.predict_classes(X_test,batch_size = batch_size)
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

Epoch 1/15
13588/13588 [==============================] - 23s 2ms/step - loss: 0.3272 - accuracy: 0.9154
Epoch 2/15
13588/13588 [==============================] - 22s 2ms/step - loss: 0.3087 - accuracy: 0.9232
Epoch 3/15
13588/13588 [==============================] - 22s 2ms/step - loss: 0.3029 - accuracy: 0.9236
Epoch 4/15
13588/13588 [==============================] - 22s 2ms/step - loss: 0.2914 - accuracy: 0.9268
Epoch 5/15
13588/13588 [==============================] - 22s 2ms/step - loss: 0.2852 - accuracy: 0.9284
Epoch 6/15
13588/13588 [==============================] - 22s 2ms/step - loss: 0.2849 - accuracy: 0.9279
Epoch 7/15
13588/13588 [==============================] - 22s 2ms/step - loss: 0.2768 - accuracy: 0.9306
Epoch 8/15
13588/13588 [==============================] - 22s 2ms/step - loss: 0.2654 - accuracy: 0.9342
Epoch 9/15
13588/13588 [==============================] - 22s 2ms/step - loss: 0.2566 - accuracy: 0.9356
Epoch 10/15
13588/13588 [==============================

# Based on the model, check the sentiment for the following two sentences

In [19]:
twt = ['He is a great leader']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=29, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0  33   5   7 144 336]]
positive


In [20]:
twt = ['He is a terrible leader']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=29, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0  33   5   7 988 336]]
negative
